In [9]:
# add path of modules
import sys
sys.path.append('d:\\Users\\serfe\\Documents\\GitHub\\seisinv\\src')

# Import local libraries
from basic_ops_seis_inv import MathOps, Filtering, Error, ConversionTool
from petro_analysis import DensityAnalysis
from post_stack import poststack_methods as ps_methods
from plot_seis_inv import PlotSeisInv

# Import standard libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [10]:
### Create instances to invoke functions/methods
math_ops = MathOps()
filtering = Filtering()
ps_methods = ps_methods()
plot_seis_inv = PlotSeisInv()
error = Error()
density_analysis = DensityAnalysis()
conversion_tool = ConversionTool()